In [1]:
import numpy
import random

In [4]:
arrives = 100000

def computeQueue(arrivals, systemCapacity): 
    N = len(arrivals)
    Q = numpy.zeros(N,dtype=int) # make a list to store the values of Q
    departures = numpy.zeros(N,dtype=int) # make a list to store the values of departures
    Q[0] = 0 #  initial queue length is 0
    for n in range(1,N):
        departures[n] = min( Q[n-1] + arrivals[n], systemCapacity[n])
        Q[n] = Q[n-1] + arrivals[n] - departures[n]

    return Q, departures

def waitingTime(arrivals,departures):
    # At[i] is the number of arrivals in time slot i
    # Convert this to the arrival time of each customer
    arrivalTimes = numpy.repeat(numpy.arange(len(arrivals)), arrivals)
    # Dt[i] is the number of departures in time slot i
    # Convert this to the departure time of each customer
    departureTimes = numpy.repeat(numpy.arange(len(departures)), departures)
    m = min(len(arrivalTimes), len(departureTimes))
    waits = departureTimes[:m] - arrivalTimes[:m]
    return waits

def baseAttention(_lambda=1, _mu=1, _arrives=100000):
    baseArrivals = numpy.random.poisson(_lambda, size=int(_arrives))
    baseCapacity = _mu*numpy.ones_like(baseArrivals)

    queueSizes, departures = computeQueue(baseArrivals, baseCapacity)
    
    baseWaitingTime = waitingTime(baseArrivals, departures)
    
    print("El tiempo medio de espera es de %f segs" % baseWaitingTime.mean())
    
    # The ones who had a queueSize == 0 from the arrivals.
    didntWait = float(queueSizes.tolist().count(0)) / float(_arrives) * 100
    print("Las solicitudes que no esperaron son el %f%%" % didntWait)

    resolutionTime = numpy.array([x + y for x, y in zip(baseWaitingTime, baseCapacity)])
    print("El tiempo de resolucion promedio es %f segs" % resolutionTime.mean())
    print("")

#
# Hago las pruebas de cada caso
#   

print("a) Arquitectura 1 - Bases de datos distribuidas \n")

p = 0.6
q = 1 - p

print("BDD1: \n")
lambdaA1 = 1.0/4.0
muA1 = 1.0/0.7
baseAttention(_lambda=lambdaA1,_mu=muA1, _arrives=int(arrives*p))

print("BDD2: \n")
lambdaA2 = 1.0/4.0
muA2 = 1.0/1.0
baseAttention(_lambda=lambdaA2,_mu=muA2, _arrives=int(arrives*q))

print("b) Arquitectura 2 - Base de datos central \n")

lambdaB = 1.0/4.0
muB = 1.0/0.8
baseAttention(_lambda=lambdaB,_mu=muB, _arrives=int(arrives))

a) Arquitectura 1 - Bases de datos distribuidas 

BDD1: 

El tiempo medio de espera es de 0.165293 segs
Las solicitudes que no esperaron son el 96.356667%
El tiempo de resolucion promedio es 1.593865 segs

BDD2: 

El tiempo medio de espera es de 0.167100 segs
Las solicitudes que no esperaron son el 96.242500%
El tiempo de resolucion promedio es 1.167100 segs

b) Arquitectura 2 - Base de datos central 

El tiempo medio de espera es de 0.172692 segs
Las solicitudes que no esperaron son el 96.186000%
El tiempo de resolucion promedio es 1.422692 segs



Conclusion: como el tiempo de resolucion de la Arquitectura 1 no es como minimo un 50% menor que el de la segunda, **le recomendamos la opcion 2.**